# Notebook for Training Model
This notebook contains all of the commands, environment setup, and code to begin training the model.

# 1. Downloading and Building Object Detection

In [1]:
!git clone https://github.com/tensorflow/models.git ../model/models

Cloning into '../model/models'...
Updating files:  66% (2112/3197)
Updating files:  67% (2142/3197)
Updating files:  68% (2174/3197)
Updating files:  69% (2206/3197)
Updating files:  70% (2238/3197)
Updating files:  71% (2270/3197)
Updating files:  72% (2302/3197)
Updating files:  73% (2334/3197)
Updating files:  74% (2366/3197)
Updating files:  75% (2398/3197)
Updating files:  76% (2430/3197)
Updating files:  77% (2462/3197)
Updating files:  78% (2494/3197)
Updating files:  79% (2526/3197)
Updating files:  80% (2558/3197)
Updating files:  81% (2590/3197)
Updating files:  82% (2622/3197)
Updating files:  83% (2654/3197)
Updating files:  84% (2686/3197)
Updating files:  85% (2718/3197)
Updating files:  86% (2750/3197)
Updating files:  87% (2782/3197)
Updating files:  88% (2814/3197)
Updating files:  89% (2846/3197)
Updating files:  90% (2878/3197)
Updating files:  91% (2910/3197)
Updating files:  92% (2942/3197)
Updating files:  93% (2974/3197)
Updating files:  94% (3006/3197)
Updating 

In [2]:
%cd ../model/models/research

C:\Users\veras\UofSC_School\ML\MilCiv\src\model\models\research


In [3]:
!protoc object_detection/protos/*.proto --python_out=.

(Next cell) Problem with "python" command may occur depending on your python verision. Try changing between python/python3 to solve

In [4]:
!copy object_detection\\packages\\tf2\\setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py
%cd ../../../notebooks

        1 file(s) copied.
Processing c:\users\veras\uofsc_school\ml\milciv\src\model\models\research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1674138 sha256=267758ee221b0fa2bd560c1aa0e73ea887b428856fd26ce252134380f9a2d579
  Stored in directory: C:\Users\veras\AppData\Local\Temp\pip-ephem-wheel-cache-8odf9kgo\wheels\14\fa\a5\0f53def934276962ac51fa4c6459e28020702dbb2ba710d9a1
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


C:\Users\veras\UofSC_School\ML\MilCiv\src\notebooks


2022-10-19 16:47:40.219090: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-19 16:47:40.219706: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-19 16:47:40.222113: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-VERA
2022-10-19 16:47:40.222207: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-VERA
Running tests under Python 3.7.13: C:\Users\veras\anaconda3\envs\aiclass\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-10-19 16:47:40.233477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlo

# 2. Generating TensorFlow Records for Data
Generating TF Records is an important step in generating the model as a whole.
This component allows TF to parse and train the model and for the distribution
of the model in a cross-platform format. See `src/notebooks/Generate TF Record` 
notebook for more detail.

In [5]:
train_csv_input = "../data/train_labels.csv"
test_csv_input = "../data/test_labels.csv"
train_output_path = "../model/train.record"
test_output_path = "../model/test.record"
image_dir = "../data/images"

In [6]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd

from tensorflow.python.framework.versions import VERSION
if VERSION >= "2.0.0a0":
    import tensorflow.compat.v1 as tf
else:
    import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == 'military tank':
        return 1
    elif row_label == 'military aircraft':
        return 2
    elif row_label == 'military truck':
        return 3
    elif row_label == 'civilian aircraft':
        return 4
    elif row_label == 'civilian car':
        return 5
    elif row_label == 'military helicopter':
        return 6
    else:
        return None
    
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)

    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_record(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    # Add all of the objects to the arrays.
    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))
        
    tf_record = tf.train.Example(features=tf.train.Features(feature = {
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes)
    }))
    
    return tf_record

In [7]:
def create_record(csv_input, output_path):
    global image_dir
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(os.getcwd(), image_dir)

    print("Reading CSV label file...")
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')

    print("Beginning compilation...")

    for group in grouped:
        tf_record = create_tf_record(group, path)
        writer.write(tf_record.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print(f"Successfully created the TFRecords: \n{output_path}")

create_record(train_csv_input, train_output_path)
create_record(test_csv_input, test_output_path)

Reading CSV label file...
Beginning compilation...
Successfully created the TFRecords: 
C:\Users\veras\UofSC_School\ML\MilCiv\src\notebooks\../model/train.record
Reading CSV label file...
Beginning compilation...
Successfully created the TFRecords: 
C:\Users\veras\UofSC_School\ML\MilCiv\src\notebooks\../model/test.record


# 3. Downloading Model and Configuration

In [8]:
%mkdir ..\model\content
%cd ../model/content

C:\Users\veras\UofSC_School\ML\MilCiv\src\model\content


First, the model itelf must be downloaded. This model is specifically used for object detection and classification, though outhers are also available.

In [10]:
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
!tar -xvf mobilenet_v2.tar.gz
!del mobilenet_v2.tar.gz

--2022-10-19 16:50:25--  http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 64.233.177.128
Connecting to download.tensorflow.org (download.tensorflow.org)|64.233.177.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8404070 (8,0M) [application/x-tar]
Saving to: 'mobilenet_v2.tar.gz.1'

     0K .......... .......... .......... .......... ..........  0% 1,97M 4s
    50K .......... .......... .......... .......... ..........  1% 4,11M 3s
   100K .......... .......... .......... .......... ..........  1% 5,03M 2s
   150K .......... .......... .......... .......... ..........  2% 10,2M 2s
   200K .......... .......... .......... .......... ..........  3% 7,91M 2s
   250K .......... .......... .......... .......... ..........  3% 3,93M 2s
   300K .......... .......... .......... .......... ..........  4% 22,5M 2s
   350K .......... .......... .........

Next, the configuration must be downloaded.

In [12]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!move ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

--2022-10-19 16:51:11--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4484 (4,4K) [text/plain]
Saving to: 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config.1'

     0K ....                                                  100%  596K=0,007s

2022-10-19 16:51:11 (596 KB/s) - 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config.1' saved [4484/4484]



        1 file(s) moved.


TensorFlow has provided a simple [script](https://blog.tensorflow.org/2021/01/custom-object-detection-in-browser.html) automatically adjusting the configuration, employed below. These values can be tweaked and adjusted for more accurate/longer training times, or for expirementation.

In [14]:
num_classes = 6
batch_size = 4
num_steps = 7500
num_eval_steps = 1000

train_record_path = "../train.record"
test_record_path = "../test.record"
model_dir = "../training"
labelmap_path = "labelmap.pbtxt"
pipeline_config_path = "mobilenet_v2.config"
fine_tune_checkpoint = "mobilenet_v2/mobilenet_v2.ckpt-1"

import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:
    config = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(labelmap_path), config)

    config = re.sub(
        'fine_tune_checkpoint: ".*?"', 
        'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), 
        config
    )
    
    config = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
        'input_path: "{}"'.format(test_record_path), config
    )
    
    config = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
        'input_path: "{}"'.format(test_record_path), config
    )
    
    config = re.sub(
        'num_classes: [0-9]+',
        'num_classes: {}'.format(6), config
    )
    
    config = re.sub(
        'batch_size: [0-9]+',
        'batch_size: {}'.format(batch_size), config
    )
    
    config = re.sub(
        'num_steps: [0-9]+',
        'num_steps: {}'.format(num_steps), config
    )
    
    f.write(config)
    print('Finished writing configuration')

Finished writing configuration


# 4. Start Training Model

In [42]:
!pip3 uninstall daal -y
!pip3 install daal==2021.4.0
!pip3 install --force-reinstall numpy==1.21.6
!pip3 install opencv-python
%cd ..

C:\Users\veras\UofSC_School\ML\MilCiv\src\model\content
Found existing installation: daal 2021.4.0
Uninstalling daal-2021.4.0:
  Successfully uninstalled daal-2021.4.0
  Using cached daal-2021.4.0-py2.py3-none-win_amd64.whl (69.0 MB)
  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
C:\Users\veras\UofSC_School\ML\MilCiv\src\model


In [64]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=content/{pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

python: can't open file 'import': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "C:\Users\veras\anaconda3\envs\aiclass\lib\site-packages\object_detection\model_lib_v2.py", line 29, in <module>
    from object_detection import eval_util
  File "C:\Users\veras\anaconda3\envs\aiclass\lib\site-packages\object_detection\eval_util.py", line 36, in <module>
    from object_detection.metrics import lvis_evaluation
  File "C:\Users\veras\anaconda3\envs\aiclass\lib\site-packages\object_detection\metrics\lvis_evaluation.py", line 23, in <module>
    from lvis import results as lvis_results
  File "C:\Users\veras\anaconda3\envs\aiclass\lib\site-packages\lvis\__init__.py", line 5, in <module>
    from lvis.vis import LVISVis
  File "C:\Users\veras\anaconda3\envs\aiclass\lib\site-packages\lvis\vis.py", line 1, in <module>
    import cv2
  Fil

In [3]:
cd ../model

/Users/wpach/Dropbox/School/USC/Fall 2022/CSCE-585/Project/src/model


In [6]:
!pwd
%reload_ext tensorboard
%ls training/train
%tensorboard --logdir=training/

/Users/wpach/Dropbox/School/USC/Fall 2022/CSCE-585/Project/src/model
events.out.tfevents.1663877095.Walters-MBP.attlocal.net.14463.0.v2
events.out.tfevents.1663877244.Walters-MBP.attlocal.net.14480.0.v2


In [ ]:
!kill 14618